# EarningsCalendarProvider

**earnings_calendar_provider.ipynb:**

EarningsCalendarProvider:  tells us when the event happens (the next earnings dates).  It’s a thin wrapper around the Finnhub API that fetches an upcoming earnings calendar for a given stock (e.g., AAPL) within a configurable lookahead window (default 120 days). You initialize EarningsCalendarProvider with an API key and ticker, it builds a Finnhub client, computes a time window from “now” to “now + lookahead,” calls earnings_calendar(...), and returns whatever Finnhub sends back. The class is meant to be a plug-in style “provider” you can swap or extend later (e.g., add a user-supplied function, multiple data sources, or logic to pick the single next earnings date).

Why EarningsCalendarProvider matters ?

Knowing the next earnings date is crucial for automation, enabling workflows like setting alerts and backtesting strategies. It also enhances user experience with timely notifications (“AAPL reports in 5 days”), calendar overlays, and conditional tasks (e.g., scrape news the day before) and facilitates data quality through

In [1]:
# import necessary libraries
from __future__ import annotations
from datetime import datetime, timedelta, timezone
from typing import Any

import os
import finnhub

class EarningsCalendarProvider:
    """
    Earnings date lookup with sensible fallbacks.
    Free Tier: 1 month of historical earnings and new updates...

    Order of operations:
      Try an optional user-supplied provider function (if given).

    A "provider function" should be:  (ticker: str, now_utc: datetime, lookahead_days: int) -> Iterable[datetime]
    It returns one or more candidate datetimes (past or future). We'll pick the next upcoming one.
    """

    def __init__(
            self,
            api_key: str,
            ticker: str,
            lookahead_days: int = 120,
            prefer_window: bool = True,
    ):
        self.client = finnhub.Client(api_key=api_key)
        self.lookahead_days = int(lookahead_days)
        self.ticker = ticker
        self.prefer_window = prefer_window  # prefer a date within lookahead window if multiple upcoming exist

    # ---------- public API ----------
    def future_earnings(self) -> Any | None:
        try:
            now = datetime.now(timezone.utc)
            window_end = now + timedelta(days=self.lookahead_days)

            # user provider
            dt = self.client.earnings_calendar(_from=now, to=window_end, symbol=self.ticker, international=False)
            if dt:
                return dt
        except Exception as e:
            print("Error fetching next_earnings error.", repr(e))

ct = EarningsCalendarProvider(ticker="AAPL", api_key="api_key_here")
calendar = ct.future_earnings()
print(f"Future earnings calendar: {calendar}")
print(calendar)

Future earnings calendar: {'earningsCalendar': [{'date': '2026-01-28', 'epsActual': None, 'epsEstimate': 2.5411, 'hour': 'amc', 'quarter': 1, 'revenueActual': None, 'revenueEstimate': 133684531371, 'symbol': 'AAPL', 'year': 2026}, {'date': '2025-10-30', 'epsActual': None, 'epsEstimate': 1.7924, 'hour': 'amc', 'quarter': 4, 'revenueActual': None, 'revenueEstimate': 103706233519, 'symbol': 'AAPL', 'year': 2025}]}
{'earningsCalendar': [{'date': '2026-01-28', 'epsActual': None, 'epsEstimate': 2.5411, 'hour': 'amc', 'quarter': 1, 'revenueActual': None, 'revenueEstimate': 133684531371, 'symbol': 'AAPL', 'year': 2026}, {'date': '2025-10-30', 'epsActual': None, 'epsEstimate': 1.7924, 'hour': 'amc', 'quarter': 4, 'revenueActual': None, 'revenueEstimate': 103706233519, 'symbol': 'AAPL', 'year': 2025}]}
